# prepare the training dataset

In [1]:
!tar czf train_data_folder.tar.gz train_data_folder

^C


In [2]:
!unzip datasets/Nest_detection.zip

unzip:  cannot find or open datasets/Nest_detection.zip, datasets/Nest_detection.zip.zip or datasets/Nest_detection.zip.ZIP.


In [3]:
!rm -rf datasets/Nest_detection/

In [4]:
!tar -xvzf datasets/Nest_detection.tar.gz -C datasets/

UnboundLocalError: cannot access local variable 'child' where it is not associated with a value

In [ ]:
"""
# rename all labels.csv files to txt
import os

rename_path = "datasets/Nest_detection/test/labels"

for filename in os.listdir(rename_path):
    # Check if the file is a .csv file
    if filename.endswith('.csv'):
        # Construct full file path
        csv_file = os.path.join(rename_path, filename)
        # Construct new file name by replacing .csv with .txt
        txt_file = os.path.join(rename_path, filename.replace('.csv', '.txt'))
        # Rename the file
        os.rename(csv_file, txt_file)
"""        

In [ ]:
"""
import pandas as pd

rename_path = "datasets/Nest_detection/test/labels"

for filename in os.listdir(rename_path):
    # Check if the file is a .csv file
    if filename.endswith('.txt'):
        label_file = os.path.join(rename_path, filename)
        df_t = pd.read_csv(label_file, sep=",")

        df_t.head()
        break
        df_t.to_csv(label_file, index=False, header=False, sep=" ")
"""


In [ ]:
!pwd

In [ ]:
!pip install ultralytics sahi 

In [ ]:
from ultralytics import YOLO
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction

In [ ]:


# Load a model
model = YOLO("yolov8l.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="datasets/data.yaml", epochs=50, imgsz=400, batch=128, device=[0, 1], patience=50)

#### from ultralytics import YOLO

model_yv8 = YOLO("yolov8l.pt")

# Train the model
results = model_yv8.train(data="/home/jovyan/deep_forest_root/data.yaml", epochs=500, imgsz=400, batch=30, patience=50, device=[ 1])

In [ ]:
from ultralytics import YOLO

model_yv8 = YOLO("yolov8l.pt")

# Train the model
results = model_yv8.train(data="/home/jovyan/deep_forest_root/data.yaml", epochs=100, imgsz=400, batch=240, patience=50, device=[0, 1])

In [ ]:
from ultralytics import YOLO

model_yv8 = YOLO("yolov8x.pt")

# Train the model
results = model_yv8.train(data="/home/jovyan/deep_forest_root/data.yaml", epochs=500, imgsz=400, batch=30, patience=50, device=[1])

## Applying Sliced Inference
https://docs.ultralytics.com/guides/sahi-tiled-inference/#import-modules-and-download-resources

In [ ]:


yolov8_model_path = "runs/detect/train24/weights/best.pt"
# yolov10_model_path = "runs/detect/train16/weights/best.pt"

detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model_path=yolov8_model_path,
    confidence_threshold=0.8,
    device="cuda:0",  # or 'cuda:0'
)

In [ ]:


result = get_sliced_prediction(
    # "deep_forest_root/jerrod_crops/Jerrod_03_21_2022DJI_0024_20.png",
    # "./deep_forest_root/JetportNew_C/JetPortNew_03_029_2022_DJI_0020.JPG",
    "./deep_forest_root/jerrod/Jerrod_03_21_2022DJI_0024.JPG",
    detection_model,
    slice_height=400,
    slice_width=400,
    overlap_height_ratio=0.1,
    overlap_width_ratio=0.1,
)

In [ ]:
from IPython.display import Image

result.export_visuals(export_dir="demo_data/")
Image("demo_data/prediction_visual.png")

In [ ]:
len(result.object_prediction_list)

In [ ]:
result.object_prediction_list

In [ ]:
threshold = 0.8
filtered_predictions = [pred for pred in result.object_prediction_list if pred.score.value >= threshold]
# 
filtered_predictions

In [ ]:
result.object_prediction_list = filtered_predictions
result.export_visuals(export_dir="demo_data/")
Image("demo_data/prediction_visual.png")

In [ ]:
# TODO predict the ground truth

In [ ]:
filtered_predictions

In [ ]:
import pandas as pd
df_pred = pd.DataFrame([x.bbox.to_voc_bbox() for x in filtered_predictions], columns=["xmin", "ymin", "xmax", "ymax"])
scores = [x.score.value for x in filtered_predictions]
df_pred["image_path"] = 'Horus_04_27_2022_DJI_0245.JPG'
df_pred["label"] = 'Nest'
df_pred["score"] = scores
df_pred

## A manual precision and recall check

In [ ]:
import os
import pandas as pd
from deepforest import main
from pathlib import Path

extract_folder = Path("./deep_forest_root/JetportNew_C")
# yolov8_model_path = "runs/detect/train20/weights/best.pt" somehow very bad model
yolov8_model_path = "runs/detect/train24/weights/best.pt"
threshold = 0.5


detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model_path=yolov8_model_path,
    confidence_threshold=threshold,
    device="cuda:0",  # or 'cuda:0'
)

predictions = []

for image_name in extract_folder.glob("*.JPG"):
    print(image_name)
    result = get_sliced_prediction(
        str(image_name),
        detection_model,
        slice_height=400,
        slice_width=400,
        overlap_height_ratio=0.1,
        overlap_width_ratio=0.1,
    )
    
    filtered_predictions = [pred for pred in result.object_prediction_list if pred.score.value >= threshold]

    df_pred = pd.DataFrame([x.bbox.to_voc_bbox() for x in filtered_predictions], columns=["xmin", "ymin", "xmax", "ymax"])
    scores = [x.score.value for x in filtered_predictions]

    df_pred["image_path"] = image_name.name
    df_pred["label"] = 'Nest'
    df_pred["score"] = scores

    predictions.append(df_pred)

df_prediction_short = pd.concat(predictions, axis=0)[["image_path", "xmin", "ymin", "xmax", "ymax", "label", "score"]]

df_prediction_short.to_csv("predictions_yolo.csv", index=False)
df_prediction_short



In [ ]:
def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union (IoU) between two bounding boxes.
    Each box is represented as [xmin, ymin, xmax, ymax]
    """
    x1_max = max(box1[0], box2[0])
    y1_max = max(box1[1], box2[1])
    x2_min = min(box1[2], box2[2])
    y2_min = min(box1[3], box2[3])

    # Calculate intersection area
    inter_width = max(0, x2_min - x1_max)
    inter_height = max(0, y2_min - y1_max)
    inter_area = inter_width * inter_height

    # Calculate areas of both bounding boxes
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Calculate union area
    union_area = box1_area + box2_area - inter_area

    # Return IoU
    if union_area == 0:
        return 0
    return inter_area / union_area

def calculate_precision_recall(ground_truth_file, predictions_file, iou_threshold=0.5, score_thresh=0.95):
    """
    Calculate precision and recall given ground truth and predictions CSV files.
    """
    # Read both CSV files
    ground_truth_df = pd.read_csv(ground_truth_file)
    predictions_df = pd.read_csv(predictions_file)

    predictions_df = predictions_df[predictions_df.score > score_thresh]
    predictions_df = predictions_df[["image_path", "xmin", "ymin", "xmax", "ymax", "label"]]
    print(f"length of predictions: {len(predictions_df)}")
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    # Loop over ground truth data by image
    for image_path in ground_truth_df['image_path'].unique():
        # Get ground truth and predictions for the current image
        ground_truth_boxes = ground_truth_df[ground_truth_df['image_path'] == image_path]
        predicted_boxes = predictions_df[predictions_df['image_path'] == image_path]

        # Track matches to avoid duplicates
        matched_ground_truth = set()
        matched_predictions = set()

        # Compare each prediction to ground truth boxes
        for idx_pred, pred_row in predicted_boxes.iterrows():
            pred_box = [pred_row['xmin'], pred_row['ymin'], pred_row['xmax'], pred_row['ymax']]
            matched = False
            for idx_gt, gt_row in ground_truth_boxes.iterrows():
                if idx_gt in matched_ground_truth:
                    continue

                gt_box = [gt_row['xmin'], gt_row['ymin'], gt_row['xmax'], gt_row['ymax']]
                iou = calculate_iou(pred_box, gt_box)

                if iou >= iou_threshold:
                    true_positives += 1
                    matched_ground_truth.add(idx_gt)
                    matched_predictions.add(idx_pred)
                    matched = True
                    break

            if not matched:
                false_positives += 1

        # Any ground truth boxes not matched are false negatives
        false_negatives += len(ground_truth_boxes) - len(matched_ground_truth)

    # Calculate precision and recall
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    return precision, recall

In [ ]:


precision, recall = calculate_precision_recall(
    ground_truth_file="/home/jovyan/deep_forest_root/JetportNew_C/ground_truth.csv", 
                           predictions_file="predictions_yolo.csv", 
                           iou_threshold=0.2, score_thresh=threshold)

print(f"precision: {precision}, recall: {recall} for the score threshold of {threshold}")
print(f"With a dataset which was trained on perfectly split data the precision is horrible now.")




In [ ]:
pd.read_csv("/home/jovyan/deep_forest_root/JetportNew_C/ground_truth.csv")

In [ ]:
pd.read_csv("predictions_yolo.csv")

In [ ]:
score_threshs = [
    0.4, 0.5, 0.6, 0.7,
    0.8, 0.9, 0.95, 0.99, 0.995, 0.999, 0.9995, 0.9999, 0.99995, 
                 0.99999, 
                 0.999999,
                 0.9999999, 
                 0.99999999, 1]

for score_thresh in score_threshs:

    precision, recall = calculate_precision_recall(
    ground_truth_file="/home/jovyan/deep_forest_root/JetportNew_C/ground_truth.csv", 
                           predictions_file="predictions_yolo.csv", 
                           iou_threshold=0.2, score_thresh=score_thresh)
    
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)
        
    print(f"score thresh: {score_thresh}, precision: {precision:.2f}, recall: {recall:.2f}, f1 score: {f1_score:.2f}")


In [ ]:
from pathlib import Path
root_folder = Path("models")

from datetime import datetime
current_date = datetime.today().strftime('%Y-%m-%d')

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f"{current_date}_{current_time}")

In [ ]:
## on the crops


In [ ]:
extract_folder = Path("./deep_forest_root/JetportNew_C_crops")
# yolov8_model_path = "runs/detect/train20/weights/best.pt"
threshold = 0.4
detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model_path=yolov8_model_path,
    confidence_threshold=threshold,
    device="cuda:0",  # or 'cuda:0'
)

predictions = []

for image_name in extract_folder.glob("*.png"):
    print(image_name)
    result = get_sliced_prediction(
        str(image_name),
        detection_model,
        slice_height=400,
        slice_width=400,
        overlap_height_ratio=0.1,
        overlap_width_ratio=0.1,
    )
    
    filtered_predictions = [pred for pred in result.object_prediction_list if pred.score.value >= threshold]

    df_pred = pd.DataFrame([x.bbox.to_voc_bbox() for x in filtered_predictions], columns=["xmin", "ymin", "xmax", "ymax"])
    scores = [x.score.value for x in filtered_predictions]

    df_pred["image_path"] = image_name.name
    df_pred["label"] = 'Nest'
    df_pred["score"] = scores

    predictions.append(df_pred)

df_prediction_short = pd.concat(predictions, axis=0)[["image_path", "xmin", "ymin", "xmax", "ymax", "label", "score"]]

df_prediction_short.to_csv(extract_folder / "predictions_yolo.csv", index=False)
df_prediction_short

In [ ]:
score_threshs = [
    0.5, 0.6, 0.7,
    0.8, 0.9, 0.95, 0.99, 0.995, 0.999, 0.9995, 0.9999, 0.99995, 
                 0.99999, 
                 0.999999,
                 0.9999999, 
                 0.99999999, 1]

for score_thresh in score_threshs:

    precision, recall = calculate_precision_recall(
    ground_truth_file="/home/jovyan/deep_forest_root/JetportNew_C_crops/gt.csv", 
                           predictions_file= extract_folder / "predictions_yolo.csv", 
                           iou_threshold=0.2, score_thresh=score_thresh)
    
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)
        
    print(f"score thresh: {score_thresh}, precision: {precision:.2f}, recall: {recall:.2f}, f1 score: {f1_score:.2f}")

## Why is YOLO so bad now??
Lets predict on data we trained on

In [ ]:
# Perform validation on data
# yolov8_model_path = "runs/detect/train20/weights/best.pt"
model = YOLO(yolov8_model_path)  
metrics = model.val(data="/home/jovyan/deep_forest_root/data.yaml")




In [ ]:
print(metrics.box.map)  # a list contains map50-95 of each category

In [ ]:
extract_folder = Path("./deep_forest_root/jerrod_crops")

threshold = 0.4
detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model_path=yolov8_model_path,
    confidence_threshold=threshold,
    device="cuda:0",  # or 'cuda:0'
)

predictions = []

for image_name in extract_folder.glob("*.png"):
    print(image_name)
    result = get_sliced_prediction(
        str(image_name),
        detection_model,
        slice_height=400,
        slice_width=400,
        overlap_height_ratio=0.1,
        overlap_width_ratio=0.1,
    )
    
    filtered_predictions = [pred for pred in result.object_prediction_list if pred.score.value >= threshold]

    df_pred = pd.DataFrame([x.bbox.to_voc_bbox() for x in filtered_predictions], columns=["xmin", "ymin", "xmax", "ymax"])
    scores = [x.score.value for x in filtered_predictions]

    df_pred["image_path"] = image_name.name
    df_pred["label"] = 'Nest'
    df_pred["score"] = scores

    predictions.append(df_pred)

df_prediction_short = pd.concat(predictions, axis=0)[["image_path", "xmin", "ymin", "xmax", "ymax", "label", "score"]]

df_prediction_short.to_csv(extract_folder / "predictions_yolo.csv", index=False)
df_prediction_short

In [ ]:
score_threshs = [
    0.4, 0.5, 0.6, 0.7,
    0.8, 0.9, 0.95, 0.99, 0.995, 0.999, 0.9995, 0.9999, 0.99995, 
                 0.99999, 
                 0.999999,
                 0.9999999, 
                 0.99999999, 1]

for score_thresh in score_threshs:

    precision, recall = calculate_precision_recall(
    ground_truth_file="/home/jovyan/deep_forest_root/jerrod_crops/gt.csv", 
                           predictions_file=extract_folder / "predictions_yolo.csv", 
                           iou_threshold=0.2, score_thresh=score_thresh)
    
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)
        
    print(f"score thresh: {score_thresh}, precision: {precision:.4f}, recall: {recall:.4f}, f1 score: {f1_score:.4f}")

In [ ]:
df_eval_pred = pd.read_csv("predictions_yolo.csv")
df_eval_pred[df_eval_pred.image_path == "Jerrod_03_21_2022DJI_0128_99.png"]

In [ ]:

df_eval_gt = pd.read_csv("/home/jovyan/deep_forest_root/jerrod_crops/gt.csv")
df_eval_gt[df_eval_gt.image_path == "Jerrod_03_21_2022DJI_0128_99.png"]